In [1]:
## 🛠️ Corrected Code Block

# --- INSTALLING LIBRARIES ---

# We can keep the library installs for a clean environment, 
# although not all are strictly needed for a text-only model.
!pip install --upgrade transformers accelerate datasets scikit-learn pyarrow


# --- IMPORTING LIBRARIES ---
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoTokenizer, RobertaModel, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
import os

RNG = 42

# --- CONFIGURATION (UPDATED NAMES) ---
FILE_PATH = '/kaggle/input/non-english-roberta/Non English reviews deleted All_Language_Data - Copy.csv'
MODEL_CHECKPOINT = "roberta-base"
MODEL_NAME = "Final_Model_Binary_RoBERTa_Base_TextOnly"
OUTPUT_DIR = f"/kaggle/working/Model_Results_Binary_Weighted_RoBERTa_TextOnly"

print(f"Loading dataset from: {FILE_PATH}")

try:
    raw_df = pd.read_csv(FILE_PATH)
except FileNotFoundError:
    print("\n❌ ERROR: File not found! Please verify the path.")
    raise

# --- SANITY CHECK (TEXT-ONLY) ---
needed_cols = [
    'reviewText', 'sentiment_label'
]

missing = [c for c in needed_cols if c not in raw_df.columns]
if missing:
    raise ValueError(f"❌ CRITICAL ERROR: Your CSV is missing these columns: {missing}")

print("--- Part 1: Setup and Data Loading Complete (Text-Only) ---")
print(f"Dataset loaded with {raw_df.shape[0]} rows.")
print(f"Target Label Source (to be binarized/filtered):\n{raw_df['sentiment_label'].value_counts().sort_index()}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.

2025-11-25 16:27:32.399805: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764088052.594285      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764088052.649743      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Loading dataset from: /kaggle/input/non-english-roberta/Non English reviews deleted All_Language_Data - Copy.csv
--- Part 1: Setup and Data Loading Complete (Text-Only) ---
Dataset loaded with 194801 rows.
Target Label Source (to be binarized/filtered):
sentiment_label
0    97036
1    97765
Name: count, dtype: int64


In [2]:
# --- PART 2: PREPROCESSING & FEATURE ENGINEERING (TEXT ONLY) ---

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight 
import numpy as np
import torch

if 'raw_df' not in globals():
    raise ValueError("❌ 'raw_df' is missing! Please run Part 1 to load the data first.")

df = raw_df.copy()

# --- CLEANING & BINARIZATION ---
print("Cleaning data...")
df = df.dropna(subset=['reviewText', 'sentiment_label']).copy() 
df['label'] = df['sentiment_label'].astype(int) 

class_names = ['Negative', 'Positive'] 
N_CLASSES = 2 
                 
# Text Prep: Use ONLY reviewText
df['combined_text'] = df['reviewText']

# --- STRATIFIED SPLIT ---
print("Splitting data (Stratified)...")
train_idx, test_idx = train_test_split(
    df.index, test_size=0.2, random_state=RNG, stratify=df['label']
)
train_df = df.loc[train_idx].reset_index(drop=True)
test_df  = df.loc[test_idx].reset_index(drop=True)

y_train = train_df['label'].values.astype(int)

# ==========================================
# CRITICAL: CALCULATE CLASS WEIGHTS
# ==========================================
print("Calculating class weights...")
classes_in_y = np.unique(y_train)

class_weights_np = compute_class_weight(
    class_weight="balanced",
    classes=classes_in_y,
    y=y_train
)
class_weights = torch.tensor(class_weights_np, dtype=torch.float)

if len(class_weights) == 1 and N_CLASSES == 2:
    print("[WARNING] Only one class found in y_train. Forcing weights to [1.0, 1.0].")
    class_weights = torch.tensor([1.0, 1.0], dtype=torch.float)

print("\n⚖️ Calculated Class Weights (Neg, Pos):")
print(class_weights)

# ==========================================
# FEATURE DIMENSIONS (FOR MODEL ARCHITECTURE) - TEXT ONLY
# ==========================================
# IMPORTANT: All auxiliary features are removed.
num_numerical_features = 0
num_categorical_features = 0
print(f"Feature Dims: Numeric={num_numerical_features}, Categorical={num_categorical_features} (Text Only)")

print("--- Part 2: Preprocessing & Weights Complete (Text-Only) ---")

Cleaning data...
Splitting data (Stratified)...
Calculating class weights...

⚖️ Calculated Class Weights (Neg, Pos):
tensor([1.0038, 0.9963])
Feature Dims: Numeric=0, Categorical=0 (Text Only)
--- Part 2: Preprocessing & Weights Complete (Text-Only) ---


In [3]:
# --- PART 3: DATASET CREATION & TOKENIZATION (TEXT ONLY) ---

from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
import numpy as np

# Define helper function (MODIFIED FOR TEXT-ONLY)
def build_features(split_df):
    out = {}
    out['combined_text'] = split_df['combined_text'].tolist()
    out['label'] = split_df['label'].astype(int).tolist()
    # Auxiliary feature logic REMOVED.
    return out

print("Building features dictionaries...")
train_feats = build_features(train_df)
test_feats  = build_features(test_df)

# HF Dataset
print("Converting to Hugging Face Datasets...")
raw_datasets = DatasetDict({
    'train': Dataset.from_dict(train_feats),
    'test':  Dataset.from_dict(test_feats),
})

# Tokenization
print(f"Downloading Tokenizer ({MODEL_CHECKPOINT})...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT) 
except Exception as e:
    print("\n❌ ERROR: Could not download tokenizer. Check internet connection.")
    raise e

def tokenize_fn(examples):
    # Truncate/pad to 256 tokens for memory efficiency
    return tokenizer(examples["combined_text"], padding="max_length", truncation=True, max_length=256)

print("Tokenizing dataset (this may take a moment)...")
tokenized_datasets = raw_datasets.map(tokenize_fn, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["combined_text"])
tokenized_datasets.set_format("torch")

print("--- Part 3: Dataset Ready & Tokenized (Text-Only) ---")

Building features dictionaries...
Converting to Hugging Face Datasets...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Tokenizing dataset (this may take a moment)...


Map:   0%|          | 0/155840 [00:00<?, ? examples/s]

Map:   0%|          | 0/38961 [00:00<?, ? examples/s]

--- Part 3: Dataset Ready & Tokenized (Text-Only) ---


In [4]:
# --- PART 4: MODEL SETUP & TRAINING (BINARY RoBERTa - TEXT ONLY) ---

import torch
import torch.nn as nn
from transformers import Trainer, TrainingArguments, RobertaModel 
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# --- MODEL ARCHITECTURE (TEXT ONLY) ---
class TextOnlyClassifier(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        # Initialize RobertaModel
        self.roberta = RobertaModel.from_pretrained(MODEL_CHECKPOINT) 
        self.dropout = nn.Dropout(0.2)
        # The classifier input size is only the RoBERTa hidden size (768 for roberta-base)
        self.classifier = nn.Linear(
            self.roberta.config.hidden_size,
            num_labels
        )
        self.num_labels = num_labels

    def forward(self, input_ids, attention_mask, labels=None):
        roberta_output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        # RoBERTa uses the first token's output for classification
        pooled_output = self.dropout(roberta_output.last_hidden_state[:, 0])

        # ONLY pooled_output is used for classification (auxiliary features removed)
        logits = self.classifier(pooled_output)

        loss = None # Loss calculation moved to WeightedTrainer

        return (loss, logits) if loss is not None else (None, logits)

# --- CUSTOM TRAINER (Handles weighted loss) ---
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        # Pass inputs to the simplified TextOnlyClassifier model
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
        logits = outputs[1]

        classifier_layer = model.module.classifier if hasattr(model, "module") else model.classifier
        weights = class_weights.to(classifier_layer.weight.device)

        # Weighted Loss Calculation
        loss_fct = nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits.view(-1, self.model.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

# --- DATA COLLATOR, METRICS (Text-Only) ---
class TextOnlyDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    def __call__(self, features):
        batch = self.tokenizer.pad(
            [{"input_ids": f["input_ids"], "attention_mask": f["attention_mask"]} for f in features],
            return_tensors="pt"
        )
        batch['labels'] = torch.tensor([f['label'] for f in features], dtype=torch.long)
        # Auxiliary feature tensors REMOVED
        return batch

def compute_metrics(pred):
    labels = pred.label_ids
    predictions = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
    preds = predictions.argmax(-1)
    
    # Use 'binary' average for the binary task
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=0) 
    acc = accuracy_score(labels, preds)
    
    return {'accuracy': acc, 'f1': f1, 'precision': prec, 'recall': rec}

# --- INIT MODEL ---
text_only_model = TextOnlyClassifier(
    num_labels=N_CLASSES
)

# Use the modified OUTPUT_DIR variable
text_only_training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=2e-5,
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    num_train_epochs=3, 
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy", 
    greater_is_better=True,
    save_total_limit=2,
    report_to="none",
    logging_steps=100,
    fp16=True 
)

text_only_trainer = WeightedTrainer(
    model=text_only_model,
    args=text_only_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=TextOnlyDataCollator(tokenizer),
    compute_metrics=compute_metrics,
)

print(f"--- Part 4: RoBERTa Base Weighted Training Setup Complete (Text-Only) ---")
print(f"Saving checkpoints to: {OUTPUT_DIR}")
print(f"Training Batch Size: {text_only_training_args.per_device_train_batch_size}")

text_only_trainer.train()
print("--- Training Complete ---")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


--- Part 4: RoBERTa Base Weighted Training Setup Complete (Text-Only) ---
Saving checkpoints to: /kaggle/working/Model_Results_Binary_Weighted_RoBERTa_TextOnly
Training Batch Size: 16


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.284600,0.297797,0.884654,0.887818,0.867167,0.909477
2,0.220600,0.292495,0.894818,0.895047,0.896424,0.893674
3,0.155800,0.307984,0.894510,0.895190,0.892732,0.897663


--- Training Complete ---


In [5]:
# --- PART 5: EVALUATION & SAVING (KAGGLE VERSION - BINARY - TEXT ONLY) ---

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import shutil 

# Use the trainer from the previous part
trainer = text_only_trainer 
tokenized_datasets = tokenized_datasets 

# =========================
# EVALUATE • REPORT • PLOTS • SAVE
# =========================
print("\n--- Final Evaluation on Test Set ---")

# 1. PLOT LOSS CURVES 
try:
    log_history = trainer.state.log_history
    train_logs = [log for log in log_history if ('loss' in log and 'eval_loss' not in log)]
    eval_logs = [log for log in log_history if ('eval_loss' in log)]

    plt.figure(figsize=(12, 5))

    # Loss Plot
    plt.subplot(1, 2, 1)
    plt.plot([log.get('epoch', i) for i, log in enumerate(train_logs, 1)], [log['loss'] for log in train_logs], label='Training Loss')
    plt.plot([log.get('epoch', i) for i, log in enumerate(eval_logs, 1)], [log['eval_loss'] for log in eval_logs], label='Validation Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Training vs Validation Loss (Text Only)'); plt.legend()

    # Accuracy Plot
    plt.subplot(1, 2, 2)
    if len(eval_logs) > 0 and 'eval_accuracy' in eval_logs[0]:
        plt.plot([log.get('epoch', i) for i, log in enumerate(eval_logs, 1)], [log['eval_accuracy'] for log in eval_logs], label='Validation Accuracy')
        plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Validation Accuracy (Text Only)'); plt.legend()
    plt.tight_layout()
    plt.savefig("loss_and_accuracy_text_only.png")
    plt.close()

except AttributeError:
    print("Skipping plots: Training history not available (trainer.train() might be commented out or failed).")


# 2. PREDICTIONS 
print("Generating predictions...")
# Note: For text-only, the arguments in predict are implicitly handled by the collator
preds_output = trainer.predict(tokenized_datasets["test"]) 

logits = preds_output.predictions[0] if isinstance(preds_output.predictions, tuple) else preds_output.predictions 

pred_labels = np.argmax(logits, axis=-1)
true_labels = np.array(tokenized_datasets["test"]["label"])

# 3. CLASSIFICATION REPORT 
print("\n--- Classification Report (BINARY RoBERTa - Text Only) ---")
report_text = classification_report(true_labels, pred_labels, target_names=class_names, digits=4, zero_division=0)
print(report_text)

# 4. CONFUSION MATRIX 
cm = confusion_matrix(true_labels, pred_labels)
row_sums = cm.sum(axis=1, keepdims=True); row_sums[row_sums == 0] = 1 
cm_norm = cm.astype('float') / row_sums

plt.figure(figsize=(12, 5))

# Counts (Integers)
plt.subplot(1, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix (Counts) (Text Only)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# Normalized (Floats with 4 decimals)
plt.subplot(1, 2, 2)
sns.heatmap(cm_norm, annot=True, fmt='.4f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix (Normalized) (Text Only)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

plt.tight_layout()
plt.savefig("confusion_matrix_text_only.png")
plt.close()

# 5. PER-CLASS BAR CHART
report_dict = classification_report(true_labels, pred_labels, target_names=class_names, output_dict=True, zero_division=0)
report_df = pd.DataFrame(report_dict).transpose()
report_df_classes = report_df.loc[[cn for cn in class_names if cn in report_df.index]]

ax = report_df_classes[['precision', 'recall', 'f1-score']].plot(kind='bar', figsize=(10, 6))
plt.title('Per-Class Metrics (BINARY RoBERTa - Text Only)')
plt.xlabel('Classes'); plt.ylabel('Score')
plt.xticks(rotation=0); plt.grid(axis='y', linestyle='--'); plt.legend(loc='lower right')
plt.savefig("per_class_metrics_text_only.png")
plt.close()

# 6. SAVE (KAGGLE SPECIFIC)
FINAL_MODEL_PATH = f"/kaggle/working/{MODEL_NAME}"

print(f"\n💾 Saving model to {FINAL_MODEL_PATH}...")
trainer.save_model(FINAL_MODEL_PATH)

# --- ZIP FOR DOWNLOAD ---
print("📦 Zipping model for easy download...")
shutil.make_archive(f"/kaggle/working/{MODEL_NAME}", 'zip', FINAL_MODEL_PATH)

print(f"✅ DONE! You can now download '{MODEL_NAME}.zip' from the 'Output' tab.")


--- Final Evaluation on Test Set ---
Generating predictions...



--- Classification Report (BINARY RoBERTa - Text Only) ---
              precision    recall  f1-score   support

    Negative     0.8932    0.8960    0.8946     19408
    Positive     0.8964    0.8937    0.8950     19553

    accuracy                         0.8948     38961
   macro avg     0.8948    0.8948    0.8948     38961
weighted avg     0.8948    0.8948    0.8948     38961


💾 Saving model to /kaggle/working/Final_Model_Binary_RoBERTa_Base_TextOnly...
📦 Zipping model for easy download...
✅ DONE! You can now download 'Final_Model_Binary_RoBERTa_Base_TextOnly.zip' from the 'Output' tab.
